# 4.1: `run_test.sh` - Decoding the `test` set using `monophones` acoustic model

`run_test.sh` 스크립트는 `decode.sh` 스크립트를 불러와서 decoding 작업을 하게 됩니다. `decode.sh` 파일에서 사용하는 `C` 명령어들에 대한 자세한 정보는 [이 곳](http://jrmeyer.github.io/asr/2016/09/12/Using-built-GMM-model-Kaldi.html)에서 확인하실 수 있습니다. 

먼저 `monophone`으로 구축한 acoustic model을 이용하여 decoding을 진행해보도록 하겠습니다. 

## Setting `kaldi_config.json`

Decoding을 시작하기 전에 `kaldi_config.json` 파일에서 `run_test` 부분을 다음과 같이 수정하겠습니다.

 - `graph_dir`=`exp/monophones/graph`  # will use the HCLG from the monophones acoustic model
 - `data_test_dir`=`data/test_dir`     # location of already-processed (during run_prepare_data.sh) test data
 - `decode_dir`=`exp/monophones/decode_test_dir`         # location to house all logs and output files
 - `num_processors`=`4`  # this will parallelize using **four** cores
 - `weight`=`10`         # default value for language modeling weight
 - `beam`=`6.0`          # default value for beam search 
 - `max_active`=`7000`   # default value for max_active nodes
 - `save_to`=`exp/experiments/default_experiment_monophones`   # location to save results and predicted transcripts
 - `non_vanilla_lda_mllt_align_hyperparameters`=`null`

In [1]:
head -n22 run_test.sh

#!/usr/bin/env bash

# This script will generate predicted transcriptions for test data found in `data/test_dir`

# ARGUMENTS
### REQUIRED
# -g <path> = full path to graph directory, default=`exp/triphones/graph/`
# -t <path> = full path to test data dir, default=`data/test_dir/`
# -d <path> = full path to experiment directory; parent directory must contain final.mdl
### OPTIONAL
# -j <int> = number of processors to use, default=2
# -w <int> = language model weight to use when returning transcription, default = `10`
# -b <float> = size of beam during graph traversal
# -m <int> = maximum number of active nodes in graph
# -q <string> = non-vanilla hyperparameters to `decode.sh` or `decode_fmllr.sh`, in the form "--beam 20"
# -z <string> = full path to experiment folder to save all important data

# OUTPUTS
# Creates one or more subdirectories in `data/test_dir/split*/` equal to setting of `-j` where
# files are copied for each parallel process
# Creates a `decode_test_dir` directory, hou

## Running `run_test.sh`

`run_test.sh` 파일을 실행하겠습니다. 앞에서 `n=250`으로 설정하였다면, 이 과정은 30분 정도 소요될 수 있습니다. 

**Note:** 중간에 다음과 같은 경고 메시지가 나올 것입니다. 

```
** split_data.sh: warning, #lines is (utt2spk,feats.scp) is (500,2620); you can 
**  use utils/fix_data_dir.sh data/test_dir/ to fix this.
** split_data.sh: warning, #lines is (utt2spk,text) is (500,2620); you can 
** use utils/fix_data_dir.sh to fix this.
```

그 이유는 우리가 전체 `utt2spk` 파일이 아니라 전체 파일의 **일부**만 사용하기 때문입니다. 위의 경고 메시지는 무시하셔도 괜찮습니다. 

In [3]:
./run_test.sh kaldi_config.json

Timestamp in HH:MM:SS (24 hour format)
01:20:41

/scratch/kaldi/egs/INSTRUCTIONAL/steps/decode.sh --model exp/monophones/final.mdl --nj 4 --beam 6.0 --max_active 7000 --scoring-opts --min-lmwt 8 --max-lmwt 12 exp/monophones/graph data/test_dir/ exp/monophones/decode_test_dir
decode.sh: feature type is delta
/scratch/kaldi/egs/INSTRUCTIONAL/steps/diagnostic/analyze_lats.sh --acwt 0.12 exp/monophones/graph exp/monophones/decode_test_dir
/scratch/kaldi/egs/INSTRUCTIONAL/steps/diagnostic/analyze_lats.sh: see stats in exp/monophones/decode_test_dir/log/analyze_alignments.log
Overall, lattice depth (10,50,90-percentile)=(3,15,59) and mean=24.5
/scratch/kaldi/egs/INSTRUCTIONAL/steps/diagnostic/analyze_lats.sh: see stats in exp/monophones/decode_test_dir/log/analyze_lattice_depth_stats.log
/scratch/kaldi/egs/INSTRUCTIONAL/steps/diagnostic/analyze_lats.sh --acwt 0.11 exp/monophones/graph exp/monophones/decode_test_dir
/scratch/kaldi/egs/INSTRUCTIONAL/steps/diagnostic/analyze_lats.sh: see stats 

실행하신 이후 다음과 같은 결과를 확인하실 수 있습니다. 

 1. `WER` vs. `SER`
   - `word error rate`는 전체 단어 중 몇 개의 단어가 **잘못** 인식되었는지를 의미합니다. `sentence error rate`는 음성 파일에서 **하나라도** 잘못 인식된 부분이 있으면 오류로 처리하고, 그 비율을 나타냅니다. 
 2. `{ws}er*`
   - **다섯** 개의 `word error rate` 파일이 생성됩니다. 뒤에 붙은 숫자는 **다섯** 종류의 weight를 사용하여 decoding을 진행하였음을 나타냅니다. 나중에 이 내용은 다시 살펴보겠습니다. 
 3. `time to decode and score`는 `decoding`에 소요된 시간을 나타냅니다. 
 4. `analyze_lat.sh` 스크립트는 반복해서 실행됩니다. 그 이유는 **다섯** 종류의 weight를 사용하기 때문입니다. 
 
`run_test.sh` 파일에서 출력하는 내용과 생성되는 파일들에 대해서는 이후 다시 살펴보도록 하겠습니다. 